In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive/'My Drive'/thunlp

/content/drive/My Drive/thunlp


In [4]:
#!git clone -b OpenKE-PyTorch https://github.com/thunlp/OpenKE

In [5]:
%cd OpenKE/openke

/content/drive/My Drive/thunlp/OpenKE/openke


In [6]:
!ls

base  config  data  __init__.py  make.sh  module  __pycache__  release


In [7]:
#!bash make.sh

In [8]:
%cd ../

/content/drive/My Drive/thunlp/OpenKE


In [8]:
#!cp icd10_KG/test_icd10.py ./

In [9]:
#!cp examples/train_transe_FB15K237.py ./

In [10]:
!ls 

benchmarks  examples  openke	 test_icd10.py
checkpoint  icd10_KG  README.md  train_transe_FB15K237.py


In [11]:
!python test_icd10.py

Input Files Path : ./icd10_KG/
The toolkit is importing datasets.
The total of relations is 1.
The total of entities is 14877.
The total of train triples is 8913.
train_dataloader model was set.
Input Files Path : ./icd10_KG/
The total of test triples is 2971.
The total of valid triples is 2971.
test_dataloader model was set.
transe model was set.
negative sampling model was set.
Finish initializing...
Epoch 999 | loss: 0.029881: 100% 1000/1000 [05:23<00:00,  3.09it/s]
checkpoint was saved.
100% 2971/2971 [00:06<00:00, 434.15it/s]
no type constraint results:
metric:			 MRR 		 MR 		 hit@10 	 hit@3  	 hit@1 
l(raw):			 0.000291 	 8414.119141 	 0.000000 	 0.000000 	 0.000000 
r(raw):			 0.006038 	 1728.562744 	 0.012790 	 0.002356 	 0.000000 
averaged(raw):		 0.003164 	 5071.340820 	 0.006395 	 0.001178 	 0.000000 

l(filter):		 0.000315 	 8408.012695 	 0.000337 	 0.000000 	 0.000000 
r(filter):		 0.006038 	 1728.562744 	 0.012790 	 0.002356 	 0.000000 
averaged(filter):	 0.003176 	 5068.

In [14]:
import openke
from openke.config import Trainer, Tester
from openke.module.model import TransE
from openke.module.loss import MarginLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader

train_dataloader = TrainDataLoader(
	in_path = "./icd10_KG/", 
	nbatches = 100,
	threads = 8, 
	sampling_mode = "normal", 
	bern_flag = 1, 
	filter_flag = 1, 
	neg_ent = 25,
	neg_rel = 0)

transe = TransE(
	ent_tot = train_dataloader.get_ent_tot(),
	rel_tot = train_dataloader.get_rel_tot(),
	dim = 200, 
	p_norm = 1, 
	norm_flag = True)

transe.load_checkpoint('./checkpoint/transe_icd10.ckpt')

In [17]:
ent_emb = transe.ent_embeddings
transe.ent_embeddings

Embedding(14877, 200)

In [18]:
rel_emb = transe.rel_embeddings
transe.rel_embeddings

Embedding(1, 200)

In [42]:
import torch

print(ent_emb.weight.shape)
print(rel_emb.weight.shape)

torch.Size([14877, 200])
torch.Size([1, 200])


In [45]:
ent_emb.weight[0] + rel_emb.weight[0]

tensor([-0.3172,  0.1350,  0.2458,  0.3384, -0.1701, -0.3131, -0.1919, -0.2593,
         0.1887,  0.4025,  0.2664,  0.2465, -0.4071,  0.2568, -0.2228, -0.1815,
         0.2189, -0.2863, -0.3790,  0.2013, -0.3535, -0.3592, -0.2467,  0.1464,
         0.1939, -0.1188, -0.3111, -0.2571, -0.3465,  0.1923, -0.2718, -0.2231,
        -0.1967, -0.2285, -0.3242, -0.2483, -0.2136, -2.4457, -0.2902, -0.0767,
         0.2236,  0.2266,  0.3390,  0.3262,  0.2847, -0.2722,  0.0810, -0.3517,
         0.3541, -0.2559,  0.2461,  0.2555,  0.3195,  0.3017, -0.2562, -0.2014,
        -0.3192,  0.3373, -0.0990, -0.2008, -0.2041,  0.2402,  0.1056, -0.1580,
        -0.2609,  0.2233,  0.2894,  0.3433, -0.2310,  0.3485,  0.2291, -0.2160,
        -0.2903,  0.3002,  0.2541,  0.3608, -0.4003, -0.3178, -0.3608,  0.1972,
        -0.2952,  0.3416,  0.1408,  0.1654,  0.2636, -0.2495,  0.1921,  0.3405,
         0.3183,  0.3361,  0.0913,  0.2335, -0.2763, -0.3229,  0.3080,  0.1751,
         0.3621, -0.2363,  0.2378,  0.19

In [26]:
ent_emb(input)[0].detach().numpy()[0:50]

array([[-0.07127835, -0.01389506,  0.01942534, ...,  0.05236193,
         0.01808841,  0.06671065],
       [-0.07127835, -0.01389506,  0.01942534, ...,  0.05236193,
         0.01808841,  0.06671065],
       [-0.07127835, -0.01389506,  0.01942534, ...,  0.05236193,
         0.01808841,  0.06671065],
       ...,
       [-0.07127835, -0.01389506,  0.01942534, ...,  0.05236193,
         0.01808841,  0.06671065],
       [-0.07127835, -0.01389506,  0.01942534, ...,  0.05236193,
         0.01808841,  0.06671065],
       [-0.07127835, -0.01389506,  0.01942534, ...,  0.05236193,
         0.01808841,  0.06671065]], dtype=float32)

In [9]:
!python train_transe_FB15K237.py

Input Files Path : ./benchmarks/FB15K237/
The toolkit is importing datasets.
The total of relations is 237.
The total of entities is 14541.
The total of train triples is 272115.
Input Files Path : ./benchmarks/FB15K237/
The total of test triples is 20466.
The total of valid triples is 17535.
Finish initializing...
Epoch 43 | loss: 4.071869:   4% 44/1000 [01:58<42:53,  2.69s/it]Traceback (most recent call last):
  File "train_transe_FB15K237.py", line 40, in <module>
    trainer.run()
  File "/content/drive/My Drive/thunlp/OpenKE/openke/config/Trainer.py", line 93, in run
    loss = self.train_one_step(data)
  File "/content/drive/My Drive/thunlp/OpenKE/openke/config/Trainer.py", line 52, in train_one_step
    loss.backward()
  File "/usr/local/lib/python3.6/dist-packages/torch/tensor.py", line 185, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/usr/local/lib/python3.6/dist-packages/torch/autograd/__init__.py", line 127, in backward
    allo

In [5]:
# import openke
# from openke.config import Trainer, Tester
# from openke.module.model import TransE
# from openke.module.loss import MarginLoss
# from openke.module.strategy import NegativeSampling
# from openke.data import TrainDataaLoader, TestDataLoader

# # dataloader for training
# train_dataloader = TrainDataLoader(
# 	in_path = "./icd10_KG/", 
# 	nbatches = 100,
# 	threads = 8, 
# 	sampling_mode = "normal", 
# 	bern_flag = 1, 
# 	filter_flag = 1, 
# 	neg_ent = 25,
# 	neg_rel = 0)

# # dataloader for test
# #test_dataloader = TestDataLoader("./benchmarks/FB15K237/", "link")

# # define the model
# transe = TransE(
# 	ent_tot = train_dataloader.get_ent_tot(),
# 	rel_tot = train_dataloader.get_rel_tot(),
# 	dim = 200, 
# 	p_norm = 1, 
# 	norm_flag = True)


# # define the loss function
# model = NegativeSampling(
# 	model = transe, 
# 	loss = MarginLoss(margin = 5.0),
# 	batch_size = train_dataloader.get_batch_size()
# )

# # train the model
# trainer = Trainer(model = model, data_loader = train_dataloader, train_times = 1000, alpha = 1.0, use_gpu = True)
# trainer.run()
# transe.save_checkpoint('./checkpoint/transe.ckpt')

# # test the model
# transe.load_checkpoint('./checkpoint/transe.ckpt')
# #tester = Tester(model = transe, data_loader = test_dataloader, use_gpu = True)
# #tester.run_link_prediction(type_constrain = False)

ModuleNotFoundError: ignored